# Gradient Descendant

In [12]:
from models import createFromBenchmark, Calculator
from math import inf
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
from utils import getAllSchedules
from math import inf

When experimenting with the gradient descendant algorithm, we will use a somewhat large dataset in order to see how the different modifications affects the performance of the algorithm

For this experiment we are using the job shop scheduling problem from banchmark_4.txt

In [13]:
# Preparing the data
# Here we are dealing with a somewhat small problem, 
# this is an ok size for testing the performance for the two different gradient descendant algorithms
filename = '/Users/ane/Projects/Performance Engineering/mini_project_4/benchmarks/benchmark_4.txt'
problem = createFromBenchmark(filename)
all_schedules = getAllSchedules(problem, set_size=3000)



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/ane/Projects/Performance Engineering/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/9w/xmv0pp696b3dv7bw9rk02fb40000gn/T/ipykernel_15418/1579138710.py", line 6, in <cell line: 6>
    all_schedules = getAllSchedules(problem, set_size=3000)
  File "/Users/ane/Projects/Performance Engineering/mini_project_4/utils.py", line 111, in getAllSchedules
    return schedules
  File "/Users/ane/Projects/Performance Engineering/mini_project_4/utils.py", line 38, in createMixedSchedule
IndexError: pop from empty list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ane/Projects/Performance Engineering/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1992, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/ane/Pr

In [14]:
# We know that the optimal schedule for this problem has a make span of 112 timeunits
iterations = 100
best = 109
calc = Calculator()

## Gradient Descendant 1
This is the simplest gradient descendant algorithm that starts with one random schedule  and checks the nearest neighbours in order to find the optimal schedule

In [15]:
count1 = 0
total_makespan1 = 0
total_iterations1 = 0

for i in range(iterations):
    best_schedule, makespan, count = calc.gradientDescendant(problem, all_schedules=all_schedules)
    total_makespan1 += makespan
    total_iterations1 += count
    if makespan <= best:
        count1 += 1

avg_makespan1 = total_makespan1/iterations
avg_iterations1 = total_iterations1/iterations
print('Out of %s times, best option was found %s times' % (iterations, count1))
print('Average makespan: ', avg_makespan1)
print('Average iterations: ', avg_iterations1)

Out of 100 times, best option was found 1 times
Average makespan:  185.7
Average iterations:  291.69


## Gradient Descendant 2

The Gradient Descendant 1 suffers from two drawbacks:  
 – It may end up in a so-called local minimum, i.e. a schedule that it is better that all
schedules of its neighborhood, but not globally optimum. A way to fix this problem
consists in starting from different initial schedules.  
 – Assume at some point of the descent, the current schedule is σ and that the algorithm
picks up a better schedule τ in the neighborhood of σ. Then, σ is in the neighborhood of
τ , which means that its score (its total processing time) must be recalculated. An obvious
waste of computation resources, that will become even more problematic in Section 2.4.  
As way to fix this problem consists in memorizing (caching) the schedules for which the
calculation of the score has been already performed. Note that this requires to store
schedules (and their scores) in a data structure in which insertion and retrieval is efficient.


The Gradient Descendant 2 starts from three different inital schedules in order to find the global minimum. The drawback from this is that it will check more schedules and calculate more timespans (which takes time). Hopefully we will save some time because we have memorized already calculated routes.

In [16]:
count2 = 0
total_makespan2 = 0
total_iterations2 = 0

print('Total iterations: ', iterations)
for i in range(iterations):
    print(i)
    best_schedule, makespan, count = calc.gradientDescendant2(problem, all_schedules=all_schedules)
    total_makespan2 += makespan
    total_iterations2 += count
    if makespan <= best:
        count2 += 1

avg_makespan2 = total_makespan2/iterations
avg_iterations2 = total_iterations2/iterations
print('Out of %s times, best option was found %s times' % (iterations, count2))
print('Average makespan: ', avg_makespan2)
print('Average iterations: ', avg_iterations2)


Total iterations:  100
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Out of 100 times, best option was found 5 times
Average makespan:  138.43
Average iterations:  877.74


As we can see, the algorithm takes longer to run than the first, but it has a better accuracy

# Gradient Descendant 3
For this algorithm we are using the Decision Tree Regressor to predict the makespan of a schedule. The reason we are using this method is to avoid calculatibng the total makespan of a schedule that possibly is worse than the best option we have already found

In regression.ipynb you can see how we ended up choosing this regression model.

The dataset we are using to train the algorithm can be found in dataframe_6.csv and consists of about 3000 different possible schedules for the job shop problem from benchmark_4 with the corresponding timespan for each schedule. We decided to save this in a csv  file in order to avoid calculating the timespan for every schedule every time as this takes a lot of time.

In [17]:
# preparing dataset
dataset = pd.read_csv('../mini_project_4/dataframes/dataframe_6.csv')
X = dataset.iloc[:, 0:64]
y = dataset.iloc[:, -1]

X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# fit the model
tree_regressor = DecisionTreeRegressor(random_state = 0)
tree_regressor.fit(X_train.values, y_train.values)

DecisionTreeRegressor(random_state=0)

In [18]:
iterations2 = 200
count3 = 0
total_makespan3 = 0
total_iterations3 = 0
best_found = inf

for i in range(iterations2):
    best_schedule, makespan, count = calc.gradientDescendant3(problem, tree_regressor, all_schedules=all_schedules)
    total_makespan3 += makespan
    total_iterations3 += count
    if makespan <= best:
        count3 += 1
    if makespan < best_found:
        best_found = makespan

avg_makespan3 = total_makespan3/iterations2
avg_iterations3 = total_iterations3/iterations2
print('Out of %s times, best option was found %s times' % (iterations2, count3))
print('Average makespan: ', avg_makespan3)
print('Average iterations: ', avg_iterations3)

Out of 200 times, best option was found 11 times
Average makespan:  160.125
Average iterations:  381.685


# Discussion

In [19]:
print('Gradient Descendant 1')
print('Out of %s times, best option was found %s times' % (iterations, count1))
print('Average makespan: ', avg_makespan1)
print('Average iterations: ', avg_iterations1)

print('\nGradient Descendant 2')
print('Out of %s times, best option was found %s times' % (iterations, count2))
print('Average makespan: ', avg_makespan2)
print('Average iterations: ', avg_iterations2)

print('\nGradient Descendant 3')
print('Out of %s times, best option was found %s times' % (iterations2, count3))
print('Best solution found: ', best_found)
print('Average makespan: ', avg_makespan3)
print('Average iterations: ', avg_iterations3)


Gradient Descendant 1
Out of 100 times, best option was found 1 times
Average makespan:  185.7
Average iterations:  291.69

Gradient Descendant 2
Out of 100 times, best option was found 5 times
Average makespan:  138.43
Average iterations:  877.74

Gradient Descendant 3
Out of 200 times, best option was found 11 times
Best solution found:  109
Average makespan:  160.125
Average iterations:  381.685


Now we can see the difference between the three algorithms. By comparing the first and the second gradient descendant algorithm, we see that the second one that start from three different initial states takes a lot longer to find the best schedule, but it also finds better schedules than the first algorithm.

By implementing a regression model to guess the score of a schedule without actually computing it we see that the running time for the algorithm has decreased a lot. The results are not as good as the Gradient Descendant 2 algorithm, but at the same time it is much better that the original algorithm.

A possible reason why the Gradient Descendant 3 algorithm doesn't find the best solution as often as the Gradient Descendant 2 is because we have not found the optimal regression model to use or that the training dataset for the model is too small. 

Because the  last algorithm is so quick it is also possible to start from maybe 6 different inital states instead of three without having a worse running time that the Gradient Descendant 2 algorithm.